# Chapitre 3: Sources de Données et Outils RAG

## Vue d'Ensemble du Système

Dans ce TP, nous travaillons avec un système RAG multi-sources pour **VéloCorp**, une entreprise de vélos. L'agent doit pouvoir accéder à différentes sources de données pour répondre aux questions.

## Les Sources de Données

### 1. **API CRM (Azure Functions)**
- **Ce que c'est** : API REST hébergée sur Azure Functions
- **Données disponibles** :
  - Opportunités commerciales (montants, statuts, commerciaux)
  - Prospects (scores, sources, taux de conversion)
  - Commerciaux (régions, quotas, performances)
  - Analytics et KPIs
- **Usage** : Questions sur les performances commerciales, pipeline, prospects

### 2. **Base de Données SQL (Azure SQL)**
- **Ce que c'est** : Base de données relationnelle SQL Server
- **Données disponibles** :
  - Données structurées (clients, commandes, produits)
  - Historique transactionnel
  - Métriques détaillées
- **Usage** : Requêtes complexes, agrégations, analyses quantitatives

### 3. **Azure Cognitive Search**
- **Ce que c'est** : Service de recherche hybride avec support keyword + vectoriel
- **Embedding model** : `sentence-transformers/all-MiniLM-L6-v2` (384 dimensions)
- **Données disponibles** :
  - Documents (guides produits, documentation technique, conversations clients)
  - Champs indexés : content, filename, type, created, document_id
  - Vecteurs d'embeddings pour recherche sémantique
- **Modes de recherche** :
  - **Keyword search** : Recherche textuelle classique (BM25)
  - **Vector search** : Recherche sémantique par similarité
  - **Hybrid search** : Combine les deux approches
- **Usage** : Questions sur les produits, documentation, recherche sémantique

### 4. **Recherche Internet (DuckDuckGo)**
- **Ce que c'est** : API de recherche web sans clé API
- **Données disponibles** : Informations publiques du web
- **Usage** : Informations actuelles, contexte externe, comparaisons marché

### 5. **Writer (Report Generator)**
- **Ce que c'est** : Outil de génération de rapports formatés
- **Formats supportés** : Markdown, HTML, Text
- **Usage** : Générer des rapports structurés, analyses, synthèses

## Comment Décrire les Outils : C'est CRUCIAL ! 🎯

La qualité de la description des outils détermine si le LLM saura les utiliser correctement.

### Anatomie d'une Bonne Description

In [ ]:
from typing import Optional
from langchain_core.tools import tool

@tool
def crm_opportunities_tool(
    status: Optional[str] = None,
    min_value: Optional[float] = None,
    sales_rep_id: Optional[int] = None
) -> str:
    """Get CRM opportunities (opportunités commerciales) from VéloCorp CRM.

    Returns opportunities with pipeline metrics (valeur_pipeline, taux_gain, etc).

    Args:
        status: Filter by status (e.g., "Proposition", "Négociation", "Gagné", "Perdu")
        min_value: Minimum opportunity value in euros
        sales_rep_id: Filter by sales rep ID

    Returns:
        CRM opportunities data with pipeline metrics
    """
    # Implementation...
    pass

### Les Éléments Clés

1. **Nom de la fonction** : Clair et descriptif (`crm_opportunities_tool`)
2. **Première ligne** : QUOI fait l'outil + contexte métier
   - ✅ "Get CRM opportunities (opportunités commerciales) from VéloCorp CRM"
   - ❌ "Get data from API"
3. **Deuxième ligne** : CE QUE ça retourne (aide le LLM à comprendre)
   - ✅ "Returns opportunities with pipeline metrics (valeur_pipeline, taux_gain, etc)"
4. **Args** : Description de CHAQUE paramètre avec EXEMPLES
   - ✅ `status: Filter by status (e.g., "Proposition", "Négociation", "Gagné", "Perdu")`
   - ❌ `status: The status`
5. **Types Python** : Utiliser `Optional`, `Literal`, etc.

### Pourquoi C'est Important ?

Le LLM utilise la description pour :
- **Décider** quel outil utiliser
- **Comprendre** quels paramètres passer
- **Savoir** quels exemples de valeurs sont valides

## Exemples d'Usage des Outils

### Exemple 1 : Outil CRM - Opportunités

**📁 Code complet** : [rag_attack/tools/api_tool.py](https://github.com/racousin/rag_attack/blob/main/rag_attack/tools/api_tool.py)

**Points pédagogiques clés** :
- Utilisation de `Optional` pour tous les paramètres de filtrage
- Exemples de valeurs dans chaque description (`e.g., "Proposition", "Négociation"`)
- Mention explicite de ce qui est retourné ("pipeline metrics")
- Contexte métier dans la première ligne ("VéloCorp CRM")

In [ ]:
# Extrait simplifié pour montrer la structure
@tool
def crm_opportunities_tool(
    status: Optional[str] = None,
    min_value: Optional[float] = None,
    sales_rep_id: Optional[int] = None
) -> str:
    """Get CRM opportunities (opportunités commerciales) from VéloCorp CRM.

    Returns opportunities with pipeline metrics (valeur_pipeline, taux_gain, etc).

    Args:
        status: Filter by status (e.g., "Proposition", "Négociation", "Gagné", "Perdu")
        min_value: Minimum opportunity value in euros
        sales_rep_id: Filter by sales rep ID

    Returns:
        CRM opportunities data with pipeline metrics
    """
    config = get_config()
    params = {}
    if status:
        params["status"] = status
    if min_value is not None:
        params["min_value"] = min_value
    if sales_rep_id is not None:
        params["sales_rep_id"] = sales_rep_id

    return azure_function_api_tool(
        config,
        endpoint="/crm/opportunites",
        params=params if params else None
    )

**Questions que cet outil peut traiter :**
- "Quelles sont les opportunités en cours de négociation ?"
  → `status="Négociation"`
- "Montre-moi les deals de plus de 5000€"
  → `min_value=5000`
- "Quelle est la valeur totale du pipeline ?"
  → Pas de filtre, agrégation dans la réponse

### Exemple 2 : Outil SQL - Requêtes Structurées

**📁 Code complet** : [rag_attack/tools/sql_tool.py](https://github.com/racousin/rag_attack/blob/main/rag_attack/tools/sql_tool.py)

**Points pédagogiques clés** :
- ⚠️ **IMPORTANT** : Mention explicite de la syntaxe T-SQL (SQL Server)
- Exemples de syntaxe dans la description (`SELECT TOP 5` au lieu de `LIMIT 5`)
- Restriction de sécurité : "SELECT only"
- Gestion du `max_rows` pour éviter les résultats trop volumineux

In [ ]:
@tool
def sql_query_tool(query: str, max_rows: int = 10) -> str:
    """Execute SQL queries against Azure SQL Server Database (T-SQL syntax).

    IMPORTANT: This is SQL Server, use T-SQL syntax:
    - Use TOP N instead of LIMIT N
    - Example: SELECT TOP 5 * FROM table ORDER BY column DESC

    Args:
        query: The SQL query to execute (SELECT only, T-SQL syntax)
        max_rows: Maximum number of rows to return (default: 10)

    Returns:
        Query results as formatted string
    """
    # Security check - only allow SELECT queries
    query_lower = query.lower().strip()
    if not query_lower.startswith("select"):
        return "Error: Only SELECT queries are allowed for safety"
    # ... rest of implementation

**Questions que cet outil peut traiter :**
- "Combien de clients avons-nous par région ?"
  → `SELECT region, COUNT(*) FROM customers GROUP BY region`
- "Top 10 des produits les plus vendus"
  → `SELECT TOP 10 product_name, SUM(quantity) FROM orders GROUP BY product_name ORDER BY SUM(quantity) DESC`

**Note importante** : La description inclut "T-SQL syntax" et "Use TOP N instead of LIMIT N" pour éviter les erreurs !

### Exemple 3 : Azure Search - Recherche Keyword

**📁 Code complet** : [rag_attack/tools/azure_search_tool.py](https://github.com/racousin/rag_attack/blob/main/rag_attack/tools/azure_search_tool.py)

**Points pédagogiques clés** :
- Recherche textuelle classique (BM25)
- Paramètre `search_fields` pour cibler des champs spécifiques
- Bonne pour les termes exacts, noms propres, SKUs

In [ ]:
@tool
def azure_search_tool(
    query: str,
    top: int = 5,
    search_fields: Optional[str] = None,
    index_name: str = "documents"
) -> str:
    """Search documents in Azure Cognitive Search.

    Args:
        query: The search query
        top: Number of results to return (default: 5)
        search_fields: Comma-separated list of fields to search
        index_name: Name of the search index (default: "documents")

    Returns:
        Formatted search results as a string
    """
    # Implementation uses Azure SDK...

**Questions que cet outil peut traiter :**
- "Quelles sont les caractéristiques du vélo électrique E-City ?"
  → `query="vélo électrique E-City"`
- "Documentation sur la garantie"
  → `query="garantie", search_fields="content,title"`

**Note** : Recherche keyword (BM25) - bonne pour les termes exacts, noms propres, SKUs.

### Exemple 3b : Azure Vector Search - Recherche Sémantique 🔍

**📁 Code complet** : [rag_attack/tools/azure_search_tool.py](https://github.com/racousin/rag_attack/blob/main/rag_attack/tools/azure_search_tool.py)

**Points pédagogiques clés** :
- ⚠️ **CRUCIAL** : Utilisation du MÊME modèle d'embedding que lors de l'indexation
- Modèle : `sentence-transformers/all-MiniLM-L6-v2` (384 dimensions)
- Nom du champ vectoriel : `"embedding"` (doit correspondre au schéma de l'index)
- Support des filtres OData pour combiner recherche sémantique + filtres structurés

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from azure.search.documents.models import VectorizedQuery

@tool
def azure_vector_search_tool(
    query: str,
    top: int = 5,
    filter: Optional[str] = None,
    index_name: str = "documents"
) -> str:
    """Perform vector search using Azure Cognitive Search with embeddings.

    Uses sentence-transformers/all-MiniLM-L6-v2 for semantic similarity search.

    Args:
        query: The search query to vectorize
        top: Number of results to return (default: 5)
        filter: OData filter expression (e.g., "type eq 'manuel'")
        index_name: Name of the search index (default: "documents")

    Returns:
        Formatted search results with relevance scores
    """
    # 1. Generate embedding using SAME model as index
    embedding_fn = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    query_embedding = embedding_fn.embed_query(query)

    # 2. Vector search using "embedding" field (384D)
    vector_query = VectorizedQuery(
        vector=query_embedding,
        k_nearest_neighbors=top,
        fields="embedding"  # ⚠️ Must match field name in index
    )
    # ... execute search

**Questions que cet outil peut traiter :**
- "Comment entretenir mon vélo ?"
  → Trouve des docs sur maintenance/entretien/nettoyage (similarité sémantique)
- "Problèmes de freinage"
  → Trouve des docs sur freins, sécurité, réparations (même si mots différents)
- "Documentation pour vélos urbains"
  → `filter="type eq 'manuel'"` pour ne chercher que dans les manuels

### Keyword vs Vector Search - Quand utiliser quoi ?

| Type de Question | Outil Recommandé | Raison |
|------------------|------------------|--------|
| "SKU-ELC-2024" | `azure_search_tool` | Terme exact |
| "vélo électrique rouge" | `azure_search_tool` | Mots-clés précis |
| "comment réparer mon vélo" | `azure_vector_search_tool` | Intention sémantique |
| "problèmes de batterie" | `azure_vector_search_tool` | Concepts similaires |
| "garantie E-City" | `azure_search_tool` | Nom propre + terme |

### ⚠️ Point Critique : Cohérence des Embeddings

Le modèle d'embedding DOIT être le même :
- ✅ Index créé avec `all-MiniLM-L6-v2` (384D) → [voir upload_azure_search.py](https://github.com/racousin/rag_attack/blob/main/scai_onepoint_rag/data/ingestion/upload_azure_search.py#L163)
- ✅ Recherche utilise `all-MiniLM-L6-v2` (384D)
- ❌ Si différents → résultats incohérents !

**Pourquoi ?** Les embeddings sont des vecteurs dans un espace spécifique à chaque modèle. Mélanger les modèles revient à comparer des pommes et des oranges.

### Exemple 4 : Recherche Web - Informations Externes

**📁 Code complet** : [rag_attack/tools/api_tool.py](https://github.com/racousin/rag_attack/blob/main/rag_attack/tools/api_tool.py)

**Points pédagogiques clés** :
- Utilisation de DuckDuckGo (pas besoin de clé API)
- Utile pour informations actuelles non présentes dans les bases internes
- Retourne titre + snippet + URL pour chaque résultat

In [ ]:
@tool
def web_search_tool(query: str, num_results: int = 5) -> str:
    """Search the web using DuckDuckGo (no API key required).

    Args:
        query: The search query
        num_results: Number of results to return

    Returns:
        Web search results
    """
    try:
        from duckduckgo_search import DDGS
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=num_results))
            # Format results...
    except ImportError:
        return "DuckDuckGo search not available. Install with: pip install duckduckgo-search"

**Questions que cet outil peut traiter :**
- "Quel temps fait-il à Toulouse aujourd'hui ?"
- "Quels sont les tendances du marché du vélo électrique en 2024 ?"
- "Comparaison prix vélos électriques concurrents"

### Exemple 5 : Report Writer - Génération de Rapports

**📁 Code complet** : [rag_attack/tools/report_writer_tool.py](https://github.com/racousin/rag_attack/blob/main/rag_attack/tools/report_writer_tool.py)

**Points pédagogiques clés** :
- Utilisation de `Literal` pour restreindre les valeurs valides de `report_type`
- Templates différents selon le type de rapport
- Support de multiples formats de sortie (markdown, HTML, text)
- Ajout automatique de métadonnées (timestamp, générateur)

In [ ]:
@tool
def report_writer_tool(
    title: str,
    content: str,
    report_type: str = "general",
    format: str = "markdown"
) -> str:
    """Generate a professional report with proper formatting.

    Args:
        title: Title of the report
        content: Main content/analysis for the report
        report_type: Type of report (general, technical, business, customer_analysis)
        format: Output format (markdown, text, html)

    Returns:
        Formatted report as a string
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    if format == "markdown":
        return _generate_markdown_report(title, content, report_type, timestamp)
    # ... autres formats

**Questions que cet outil peut traiter :**
- "Génère un rapport d'analyse des ventes du Q4"
  → `title="Analyse Ventes Q4", content="...", report_type="business"`
- "Crée un rapport technique sur les performances"
  → `report_type="technical", format="markdown"`

## Principes de Conception des Outils

### 1. **Un Outil = Une Responsabilité**
- Ne pas créer un outil qui fait "tout"
- Préférer plusieurs outils spécialisés
- Exemple : `crm_opportunities_tool` vs `crm_prospects_tool` plutôt qu'un seul `crm_tool`

### 2. **Paramètres Optionnels pour la Flexibilité**

In [ ]:
def my_tool(
    required_param: str,           # Toujours nécessaire
    optional_filter: Optional[str] = None  # Flexibilité
):
    pass

### 3. **Exemples dans les Descriptions**
- Le LLM apprend par exemple
- Inclure des valeurs réalistes
- Format : `(e.g., "value1", "value2", "value3")`

### 4. **Gestion des Erreurs**

In [ ]:
try:
    # Logic
    return result
except Exception as e:
    return f"Error: {str(e)}"  # Retourner l'erreur comme string

### 5. **Format de Retour Cohérent**
- Toujours retourner des strings
- JSON formaté avec `json.dumps(..., indent=2)`
- Messages d'erreur clairs

## Flux de Décision du LLM

```
Question Utilisateur
    ↓
LLM analyse la question
    ↓
LLM lit les descriptions des outils
    ↓
LLM choisit le(s) outil(s) approprié(s)
    ↓
LLM génère les paramètres
    ↓
Exécution de l'outil
    ↓
LLM reçoit le résultat
    ↓
LLM peut :
    - Utiliser un autre outil
    - Formuler la réponse finale
```

## Questions de Réflexion

1. **Pourquoi séparer les outils CRM en plusieurs fonctions** (`crm_opportunities_tool`, `crm_prospects_tool`, etc.) au lieu d'un seul `crm_tool` ?

2. **Pourquoi inclure des exemples de valeurs** dans les descriptions de paramètres ?

3. **Que se passe-t-il si on ne spécifie pas que la base SQL utilise T-SQL** (et non PostgreSQL) ?

4. **Comment le LLM décide-t-il entre utiliser `azure_search_tool` vs `web_search_tool`** pour une question sur les produits ?

5. **Pourquoi retourner des strings plutôt que des objets Python** depuis les outils ?

6. **Que se passe-t-il si le modèle d'embedding utilisé pour la recherche est différent de celui utilisé lors de l'indexation** ? (par exemple, utiliser `all-MiniLM-L6-v2` pour l'index mais `text-embedding-ada-002` pour la recherche)

## Comparaison : Bonne vs Mauvaise Description

### ❌ Mauvaise Description

In [ ]:
@tool
def get_data(param: str) -> str:
    """Get data"""
    pass

**Problèmes** :
- Nom vague
- Pas de contexte
- Pas d'exemples
- Le LLM ne saura pas quand l'utiliser

### ✅ Bonne Description

In [ ]:
@tool
def crm_prospects_tool(
    status: Optional[str] = None,
    min_score: Optional[int] = None,
    source: Optional[str] = None,
    limit: int = 50
) -> str:
    """Get CRM prospects from VéloCorp CRM.

    Returns prospects with summary stats (total, qualified, conversion rate, etc).

    Args:
        status: Filter by prospect status (e.g., "Nouveau", "Qualifié", "Contacté")
        min_score: Minimum lead score (0-100)
        source: Filter by lead source (e.g., "Site web", "Salon", "Référence")
        limit: Maximum number of prospects to return (default: 50)

    Returns:
        CRM prospects data with conversion metrics
    """
    pass

**Avantages** :
- Nom clair et contextualisé
- Indique CE QUI est retourné
- Exemples de valeurs pour chaque paramètre
- Le LLM comprend quand et comment l'utiliser

## Résumé

**Ce chapitre couvre** :
- ✅ Les 5 sources de données (API, SQL, Search, Web, Writer)
- ✅ Comment décrire les outils de manière efficace
- ✅ Exemples d'usage pour chaque type d'outil
- ✅ Principes de conception des outils

**Points clés à retenir** :
1. La description de l'outil est AUSSI importante que son implémentation
2. Inclure des exemples de valeurs dans les paramètres
3. Un outil = une responsabilité claire
4. Toujours retourner des strings formatées
5. Gérer les erreurs explicitement
6. Pour la recherche vectorielle : utiliser le MÊME modèle d'embedding que lors de l'indexation

**Prochaine étape** : Nous allons combiner ces outils avec LangGraph pour créer un agent RAG complet capable de répondre à des questions complexes nécessitant plusieurs sources !

---

**📚 Ressources** :
- [Code source complet sur GitHub](https://github.com/racousin/rag_attack)
- [Tools directory](https://github.com/racousin/rag_attack/tree/main/rag_attack/tools)
- [Data ingestion scripts](https://github.com/racousin/rag_attack/tree/main/scai_onepoint_rag/data/ingestion)